In [1]:
import sys
sys.path.insert(0, '../CClassCSemigroups/')

In [2]:
import CsemigroupsCPP as CPP

In [3]:
import numpy as np

In [4]:
import itertools

In [5]:
generators=[[3,0],[4,0],[5,0],[3,1],[4,1],[5,1],[2,2],[3,2],[3,3],[4,3]]
cs = CPP.Csemigroup(generators)
print("Los huecos son:",cs.GetGaps())
print(cs.GetHyperplanes())
cs.GetGenerators()

Los huecos son: [[1, 0], [1, 1], [2, 0], [2, 1], [4, 2]]
[[0, 1], [1, -1]]


[[3, 0],
 [4, 0],
 [5, 0],
 [3, 1],
 [4, 1],
 [5, 1],
 [2, 2],
 [3, 2],
 [3, 3],
 [4, 3]]

In [6]:
def ComputeCone(bound,eq):
    cone = []
    it = itertools.product(*[range(i+1) for i in bound])
    for x in it:
        if CPP.belongCone(list(x),eq):
            cone.append(list(x))
    return cone
cono = ComputeCone([8,4],[[0, 1], [1, -1]])

In [7]:
def PrecGradLex(a,b): # Check if A<B
    sumA = np.array(a).sum()
    sumB = np.array(b).sum()
    if sumA < sumB:
        return True
    elif sumA > sumB:
        return False
    else:
        c = np.array(b)-np.array(a)
        for x in c:
            if x > 0:
                return True
            if x < 0:
            
                return False
# PrecGradLex([4,0],[4,2])

In [8]:
def PrecGradRevLex(a,b): # Check if A<B
    sumA = np.array(a).sum()
    sumB = np.array(b).sum()
    if sumA < sumB:
        return True
    elif sumA > sumB:
        return False
    else:
        c = np.array(b)-np.array(a)
        for x in c:
            if x < 0:
                return True
            if x > 0:
                return False

In [9]:
def MaximumOrder(v,order):
    maximum = v[0]
    for x in v:
        if order(maximum,x):
            maximum = x
    return maximum
MaximumOrder([[1, 0], [2, 1], [2, 2], [4, 2], [3, 1]],PrecGradRevLex)

[4, 2]

In [10]:
def GetSemigroupFromGaps(gaps,eq,order): # Note: Last gap is the frobenius element.
    frob = MaximumOrder(gaps,order)
    bound =  [3*frob[i] for i in range(len(frob))]### <--- COMPROBAR SI ESTA COTA SE PUEDE MEJORAR!!!
    cone = []
    it = itertools.product(*[range(i+1) for i in bound])
    for x in it:
        if CPP.belongCone(list(x),eq) and not CPP.allZero(list(x)):
            cone.append(list(x))
    for x in gaps:
        cone.remove(x)
    cs = CPP.Csemigroup(cone)
    return cs
# GetSemigroupFromGaps([[1, 0], [2, 0], [1, 1], [2, 1],[4,2]],[[0, 1], [1, -1]])

In [11]:
sg_aux = GetSemigroupFromGaps([[1, 0], [2, 1], [2, 2], [4, 2], [3, 1]],[[0, 1], [1, -1]],PrecGradLex)
print(sg_aux.GetGaps())
print(sg_aux.GetGenerators())

[[1, 0], [2, 1]]
[[2, 0], [3, 0], [1, 1], [3, 2]]


In [12]:
def ComputeIC(cone,frob,eq):
    aux = []
    for x in cone:
        if CPP.belongCone([frob[i]-x[i] for i in range(len(x))],eq):
            aux.append(x)
    return aux
IC = ComputeIC(cono,[4,2],[[0, 1], [1, -1]])

In [13]:
def ComputeRootTreeIrreductible(IC, frob, order,eq):
    hFrob = [x/2 for x in frob]
    i1 = []
    for x in IC:
        if not CPP.allZero(list(x)) and not order(hFrob,x):
            i1.append(x)
    i1.append(frob)
    return GetSemigroupFromGaps(i1,eq,order)
ComputeRootTreeIrreductible(IC, [4,2],PrecGradLex,[[0, 1], [1, -1]]).GetGenerators()

[[3, 0],
 [4, 0],
 [5, 0],
 [3, 1],
 [4, 1],
 [5, 1],
 [2, 2],
 [3, 2],
 [3, 3],
 [4, 3]]

In [14]:
def GetGeneratorsBetweenHalfFrobAndFrob(gen, frob,order):
    hFrob = [x/2 for x in frob]
    i2 = []
    for x in gen:
        aux = list(np.array(frob)-np.array(x))
        if order(hFrob,x) and CPP.allPositive([frob[i]-x[i] for i in range(len(x))]):
            i2.append(x)
    return i2
GetGeneratorsBetweenHalfFrobAndFrob(generators,[4,2],PrecGradLex)

[[3, 0], [4, 0], [3, 1], [4, 1], [2, 2], [3, 2]]

In [15]:
def CheckPropertiesGeneratorsTree(gen, frob,IC,order):
    aux = []
    candidates = GetGeneratorsBetweenHalfFrobAndFrob(gen, frob,order)
    #sprint(candidates)
    for x in candidates:
        #print("x =",x)
        #print("x in IC",x in IC)
        #print("2x-f not in S",not CPP.belongByGens([2*x[i]-frob[i] for i in range(len(x))],gen))
        if x in IC and not CPP.belongByGens([2*x[i]-frob[i] for i in range(len(x))],gen) and [3*x[i] for i in range(len(x))] != [2*frob[i] for i in range(len(x))] and [4*x[i] for i in range(len(x))] != [3*frob[i] for i in range(len(x))]:
            aux.append(x)
    return aux
CheckPropertiesGeneratorsTree([[3, 0], [4, 0], [5, 0], [1, 1], [3, 2], [5, 3]],[4,2],IC,PrecGradLex)

[]

In [16]:
def ComputeIrreducibleChildren(sg,frob,order, ic,eq):
    children = []
    A = CheckPropertiesGeneratorsTree(sg.GetGenerators(),frob,ic,order)
    print("A =",A)
    #print(sg.GetGenerators())
    for x in A:
        gaps = list(sg.GetGaps())
        gaps.append(x)
        gaps.remove([frob[i]-x[i] for i in range(len(x))])
        children.append(GetSemigroupFromGaps(gaps,eq,order))
    return children

sg_aux = CPP.Csemigroup([[3, 0], [4, 0], [5, 0], [1, 1], [3, 2], [5, 3]])
print(sg_aux.GetGaps())
kk = ComputeIrreducibleChildren(sg_aux,[4,2],PrecGradLex,IC,[[0, 1], [1, -1]])
for x in kk:
    print(x.GetGaps())

[[1, 0], [2, 0], [2, 1], [3, 1], [4, 2]]
A = []


In [17]:
IC = [[0, 0], [1, 0], [1, 1], [2, 0], [2, 1], [2, 2], [3, 1], [3, 2], [4, 2]]

In [18]:
GetGeneratorsBetweenHalfFrobAndFrob(IC, [4,2],PrecGradLex)

[[2, 2], [3, 1], [3, 2], [4, 2]]

In [19]:
def ComputeTreeIrreducible(frob, eq, order):
    bound =  [3*frob[i] for i in range(len(frob))]### <--- COMPROBAR SI ESTA COTA SE PUEDE MEJORAR!!!
    cone = ComputeCone(bound,eq)
    sf = ComputeRootTreeIrreductible(cone, frob, order,eq)
    ic = ComputeIC(cone,frob,eq)
    print(sf.GetGenerators())
    X = [sf]
    Y = []
    while X != []:
        print(len(X))
        for sg in X:
            print("Huecos padre:",sg.GetGaps())
            print("Huecos hijos:")
            children = ComputeIrreducibleChildren(sg,frob,order, ic,eq)
            for kk in children:
                print(kk.GetGaps())
            Y.append([sg,children])
        X = children
        
        print("En X hay:")
        for kk in X:
            print(kk.GetGenerators())
        print("\n")
    return(Y)

arbol  = ComputeTreeIrreducible([4,2], [[0, 1], [1, -1]], PrecGradLex)

[[3, 0], [4, 0], [5, 0], [3, 1], [4, 1], [5, 1], [2, 2], [3, 2], [3, 3], [4, 3]]
1
Huecos padre: [[1, 0], [1, 1], [2, 0], [2, 1], [4, 2]]
Huecos hijos:
A = [[3, 1], [2, 2]]
[[1, 0], [2, 0], [2, 1], [3, 1], [4, 2]]
[[1, 0], [1, 1], [2, 1], [2, 2], [4, 2]]
En X hay:
[[3, 0], [4, 0], [5, 0], [1, 1], [3, 2], [5, 3]]
[[2, 0], [3, 0], [3, 1], [4, 1], [3, 2], [3, 3], [4, 3], [4, 4], [5, 4], [5, 5]]


2
Huecos padre: [[1, 0], [2, 0], [2, 1], [3, 1], [4, 2]]
Huecos hijos:
A = []
Huecos padre: [[1, 0], [1, 1], [2, 1], [2, 2], [4, 2]]
Huecos hijos:
A = [[3, 2]]
[[1, 1], [2, 1], [2, 2], [3, 2], [4, 2]]
En X hay:
[[1, 0], [3, 1], [5, 2], [3, 3], [4, 4], [5, 5]]


1
Huecos padre: [[1, 1], [2, 1], [2, 2], [3, 2], [4, 2]]
Huecos hijos:
A = []
En X hay:




In [20]:
for x in arbol:
    print("Padre",x[0].GetGenerators())
    print("Hijos:")
    for y in x[1]:
        print("   ->",y.GetGenerators(),len(y.GetGaps()))

Padre [[3, 0], [4, 0], [5, 0], [3, 1], [4, 1], [5, 1], [2, 2], [3, 2], [3, 3], [4, 3]]
Hijos:
   -> [[3, 0], [4, 0], [5, 0], [1, 1], [3, 2], [5, 3]] 5
   -> [[2, 0], [3, 0], [3, 1], [4, 1], [3, 2], [3, 3], [4, 3], [4, 4], [5, 4], [5, 5]] 5
Padre [[3, 0], [4, 0], [5, 0], [1, 1], [3, 2], [5, 3]]
Hijos:
Padre [[2, 0], [3, 0], [3, 1], [4, 1], [3, 2], [3, 3], [4, 3], [4, 4], [5, 4], [5, 5]]
Hijos:
   -> [[1, 0], [3, 1], [5, 2], [3, 3], [4, 4], [5, 5]] 5
Padre [[1, 0], [3, 1], [5, 2], [3, 3], [4, 4], [5, 5]]
Hijos:


In [21]:
arbol2  = ComputeTreeIrreducible([4,2], [[0, 1], [1, -1]], PrecGradRevLex)

[[4, 0], [5, 0], [6, 0], [7, 0], [3, 1], [4, 1], [5, 1], [6, 1], [2, 2], [3, 2], [5, 2], [3, 3], [4, 3]]
1
Huecos padre: [[1, 0], [1, 1], [2, 0], [2, 1], [3, 0], [4, 2]]
Huecos hijos:
A = [[3, 1], [2, 2]]
[[1, 0], [2, 0], [2, 1], [3, 0], [3, 1], [4, 2]]
[[1, 0], [1, 1], [2, 1], [2, 2], [3, 0], [4, 2]]
En X hay:
[[4, 0], [5, 0], [6, 0], [7, 0], [1, 1], [4, 1], [3, 2], [5, 3]]
[[2, 0], [5, 0], [3, 1], [4, 1], [3, 2], [3, 3], [4, 3], [4, 4], [5, 4], [5, 5]]


2
Huecos padre: [[1, 0], [2, 0], [2, 1], [3, 0], [3, 1], [4, 2]]
Huecos hijos:
A = []
Huecos padre: [[1, 0], [1, 1], [2, 1], [2, 2], [3, 0], [4, 2]]
Huecos hijos:
A = [[3, 2]]
[[1, 1], [2, 1], [2, 2], [3, 2], [4, 2]]
En X hay:
[[1, 0], [3, 1], [5, 2], [3, 3], [4, 4], [5, 5]]


1
Huecos padre: [[1, 1], [2, 1], [2, 2], [3, 2], [4, 2]]
Huecos hijos:
A = []
En X hay:


